# Matrix Factorization


## Khởi tạo và chuẩn hóa ma trận


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse


class MF(object):
    """docstring for CF"""

    def __init__(self, Y_data, K, lam=0.1, Xinit=None, Winit=None,
                learning_rate=0.5, max_iter=1000, print_every=100, user_based=1):
        self.Y_raw_data = Y_data
        self.K = K
        # Tham số chính quy
        self.lam = lam
        # Tỉ lệ học
        self.learning_rate = learning_rate
        # Tối đa số lần lặp
        self.max_iter = max_iter
        # In kết quả sau lần lặp print_every
        self.print_every = print_every
        self.user_based = user_based
        # Số lượng người dùng và items, thêm 1 vì id bắt đầu từ 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1
        self.n_items = int(np.max(Y_data[:, 1])) + 1

        if Xinit is None:
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit

        if Winit is None:
            self.W = np.random.randn(K, self.n_users)
        else:
            self.W = Winit

        self.n_ratings = Y_data.shape[0]
        # Chuẩn hóa dữ liệu
        self.Y_data_n = self.Y_raw_data.copy()

    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        # Chuẩn hóa dựa trên mục, chỉ cần chuyển hai cột dữ liệu đầu tiên
        else:
            user_col = 1
            item_col = 0
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col]
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # Chỉ số xếp hạng hàng được thực hiện bởi người dùng n
            # Chuyển đổi các chỉ số phải là số nguyên
            ids = np.where(users == n)[0].astype(np.int32)
            # Chỉ số của tất cả các xếp hạng được liên kết với người dùng n
            item_ids = self.Y_data_n[ids, item_col]
            # Xếp hạng tương ứng
            ratings = self.Y_data_n[ids, 2]
            # Tính kết quả trung bình của ratings
            m = np.mean(ratings)
            if np.isnan(m):
                m = 0  # Gán m = 0 để tránh giá trị rỗng và NaN
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]

    def loss(self):
        L = 0
        for i in range(self.Y_data_n.shape[0]):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(
                self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2

        # Chính quy hóa
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') +
                        np.linalg.norm(self.W, 'fro'))
        return L

    def get_items_rated_by_user(self, user_id):
        """
        Nhận tất cả các mục được người dùng n xếp hạng và xếp hạng tương ứng
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # Chỉ số mục do user_id xếp hạng
        # Cần phải + 1 tới user_id vì trong rate_matrix id được bắt đầu bằng 1
        # Chỉ mục trong python được tính từ 0
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        item_ids = self.Y_data_n[ids, 1].astype(
            np.int32)  # index starts from 0
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)

    def get_users_who_rate_item(self, item_id):
        """
        Nhận tất cả users đánh giá item m và xếp hạng tương ứng
        """
        ids = np.where(self.Y_data_n[:, 1] == item_id)[0]
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)

    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))

    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))

    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print('iter =', it + 1, ', loss =',
                    self.loss(), ', RMSE train =', rmse_train)

    def pred(self, u, i):
        """ 
        Dự đoán xếp hạng của người dùng u cho item i
        """
        u = int(u)
        i = int(i)

        if self.user_based:
            bias = self.mu[u]
        else:
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias
        if pred < 1:
            return 1
        if pred > 5:
            return 5
        return pred

    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()

        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings = []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))

        return predicted_ratings

    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0  # Lỗi bình phương
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2

        RMSE = np.sqrt(SE/n_tests)
        return RMSE


r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep=' ', names=r_cols, encoding='latin-1')
Y_data = ratings.values


rs = MF(Y_data, K=2, max_iter=1000, print_every=1000)

rs.fit()
rs.pred(6, 1)


iter = 1000 , loss = 0.5093660341261714 , RMSE train = 0.7971830482859001


2.933193027054375

In [2]:
print(rs.X.dot(rs.W) + rs.mu)


[[4.1656273  4.01700146 1.80621594 0.2203071  1.71805744 1.43203371
  2.36978586]
 [3.65928493 3.29021224 2.21260546 0.81677516 2.1444647  1.47801122
  2.93319303]
 [4.0027105  3.85436697 1.87505243 0.46408714 1.87162249 1.42396211
  2.46778697]
 [2.44440578 1.69636391 3.05727647 2.35711016 3.20201973 1.54017815
  4.10962934]
 [2.25733488 1.32169047 3.29972566 2.5001616  3.33515857 1.59125028
  4.44192582]]


# Áp dụng lên MovieLens 100k


In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t',
                           names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t',
                           names=r_cols, encoding='latin-1')

rate_train = ratings_base.values
rate_test = ratings_test.values

# Chỉ số bắt đầu từ 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1


In [4]:
rs = MF(rate_train, K=10, lam=.1, print_every=10,
        learning_rate=0.75, max_iter=100, user_based=1)
rs.fit()
# Đánh giá trên dữ liệu thử nghiệm
RMSE = rs.evaluate_RMSE(rate_test)
print('\nUser-based MF, RMSE =', RMSE)


iter = 10 , loss = 5.623838819820887 , RMSE train = 1.1964447545558297
iter = 20 , loss = 2.631475900420379 , RMSE train = 1.0376697790632379
iter = 30 , loss = 1.33993718076647 , RMSE train = 1.02957166403085
iter = 40 , loss = 0.7512801947384818 , RMSE train = 1.0292248283935725
iter = 50 , loss = 0.4814993207408244 , RMSE train = 1.0292133412902826
iter = 60 , loss = 0.35779034319396386 , RMSE train = 1.0292139102732973
iter = 70 , loss = 0.30105984559137006 , RMSE train = 1.0292142181448982
iter = 80 , loss = 0.2750441884405376 , RMSE train = 1.0292143030362113
iter = 90 , loss = 0.26311383315258896 , RMSE train = 1.0292143242657077
iter = 100 , loss = 0.2576427655947336 , RMSE train = 1.0292143294023373

User-based MF, RMSE = 1.0603799055180005


In [5]:
rs = MF(rate_train, K=10, lam=.1, print_every=10,
        learning_rate=0.75, max_iter=100, user_based=0)
rs.fit()
# Đánh giá trên dữ liệu thử nghiệm
RMSE = rs.evaluate_RMSE(rate_test)
print('\nItem-based MF, RMSE =', RMSE)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iter = 10 , loss = 5.627300815677279 , RMSE train = 1.1759647866308531
iter = 20 , loss = 2.6193741595966102 , RMSE train = 1.0052778973877226
iter = 30 , loss = 1.3250750046123816 , RMSE train = 0.9965435064507597
iter = 40 , loss = 0.7352902392363744 , RMSE train = 0.9961867781427354
iter = 50 , loss = 0.4649860406314433 , RMSE train = 0.9961787000181426
iter = 60 , loss = 0.34103390141671786 , RMSE train = 0.9961802195401405
iter = 70 , loss = 0.28419100621754856 , RMSE train = 0.9961807357151344
iter = 80 , loss = 0.2581235473766677 , RMSE train = 0.9961808643812019
iter = 90 , loss = 0.24616935118952127 , RMSE train = 0.9961808947310101
iter = 100 , loss = 0.2406873183932131 , RMSE train = 0.9961809017643232

Item-based MF, RMSE = 1.0486424539909174


In [6]:
rs = MF(rate_train, K=2, lam=0, print_every=10,
        learning_rate=1, max_iter=100, user_based=0)
rs.fit()
# Đánh giá trên dữ liệu thử nghiệm
RMSE = rs.evaluate_RMSE(rate_test)
print('\nItem-based MF, RMSE =', RMSE)


iter = 10 , loss = 1.0829562408750968 , RMSE train = 1.4212725301333173
iter = 20 , loss = 1.0262379415927207 , RMSE train = 1.4055257721440935
iter = 30 , loss = 0.9755228320005906 , RMSE train = 1.3905237849533638
iter = 40 , loss = 0.9299669098542266 , RMSE train = 1.376209933862853
iter = 50 , loss = 0.888871471060061 , RMSE train = 1.362594825251479
iter = 60 , loss = 0.8516538552854371 , RMSE train = 1.349657297603167
iter = 70 , loss = 0.8178248796755295 , RMSE train = 1.3373137448648758
iter = 80 , loss = 0.78697126033549 , RMSE train = 1.325564912472822
iter = 90 , loss = 0.7587417944789745 , RMSE train = 1.3143856325750538
iter = 100 , loss = 0.7328364069464384 , RMSE train = 1.3037201004671581

Item-based MF, RMSE = 1.3895662847193322


In [7]:
RMSE = rs.evaluate_RMSE(rate_train)


In [8]:
print(RMSE)


1.3037201004671581


# MovieLens 1M


In [9]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv(
    'ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')


/var/folders/mx/__zp2dyx02g_k1_g_3n5xcrc0000gn/T/ipykernel_34684/3273828672.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_base = pd.read_csv('ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')


In [1]:
from sklearn.model_selection import train_test_split
ratings = ratings_base.values
ratings[:, :2] -= 1


rate_train, rate_test = train_test_split(
    ratings, test_size=0.33, random_state=42)
print(rate_train.shape, rate_test.shape)

rs = MF(rate_train, K=2, lam=0.1, print_every=2,
        learning_rate=2, max_iter=10, user_based=0)
rs.fit()
# Đánh giá trên dữ liệu thử nghiệm
RMSE = rs.evaluate_RMSE(rate_test)
print('\nItem-based MF, RMSE =', RMSE)


NameError: name 'ratings_base' is not defined